In [1]:
import os 


In [2]:
# since notebook wd is /home/pistachio/pistachio/research/trials.ipynb, changning 
# wd to E:\Pistachio_Classification_Project
from pathlib import Path
os.chdir(Path("E:/Pistachio_Classification_Project"))

In [3]:
%pwd

'E:\\Pistachio_Classification_Project'

###  3. Update Entity 
**Enity is a return type of a function**. 
if a function does not have any particular return type, we can create a custom return type in the entity 

In [4]:
from dataclasses import dataclass

# refer config.yaml file, below we are defining a return type
#1. defining entity
@dataclass(frozen=True) # used to create classes that are primarily meant to store data or to return certain type of objects
class DataIngestionConfig:
    root_dir :Path
    source_URL: str
    local_data_file : Path
    unzip_dir:Path

In [5]:
# read the yaml file
from cnnClassifier.constants.__init__ import PARAMS_FILE_PATH,CONFIG_FILE_PATH 
from cnnClassifier.utils.common import read_yaml, create_directories

### 4. update configuration manager in src config

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config= read_yaml(config_file_path) # return ConfigBox object with 'artifacts_root' as a key name
        self.params= read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig: # see two cells above for DataIngestionConfig 
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

### 5. Updating the Components

In [7]:
import os
import urllib.request as request # download the dataset from the URL
import zipfile
from cnnClassifier import logger
# from cnnClassifier.utils.common import

In [8]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig): # get_data_ingestion_config
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve (
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {filename}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  

### DataIngestion PipeLine

In [9]:
try:
    conf_manager = ConfigurationManager()
    DataIngestionConfig = conf_manager.get_data_ingestion_config()
    DI = DataIngestion(DataIngestionConfig)
    DI.download_file()
    DI.extract_zip_file()
except Exception as e:
    raise e

[2023-11-01 08:55:19,762: INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-11-01 08:55:19,768: INFO : common : yaml file: params.yaml loaded successfully]
[2023-11-01 08:55:19,770: INFO : common : created directory at: artifacts]
[2023-11-01 08:55:19,771: INFO : common : created directory at: artifacts/data_ingestion]
[2023-11-01 08:55:20,250: INFO : 2729816172 : Downloaded artifacts/data_ingestion/archieve.zip]


BadZipFile: File is not a zip file